# Домашнее задание 6. Python для анализа данных: pandas

In [1]:
import pandas as pd
from datetime import datetime
import re

## Задание 1
Используем файл keywords.csv.
 
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
 
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток в geo_data.

Результат классификации запишите в отдельный столбец region.
 

In [2]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [3]:
keywords = pd.read_csv('../data/keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [4]:
city_code = {value: key for key, values in geo_data.items() for value in values}

In [5]:
def get_region_code(value):
    for key, items in geo_data.items():
        for i in value.split():
            if i in items:
                return key
    
def set_region_code(value):
    df =value 
    df['region_code'] = df['keyword'].apply(lambda row: get_region_code(row))
    return df
    
new_df = set_region_code(keywords)
new_df[(new_df['region_code'].notna()==True)].head(10)

,keyword,shows,region_code
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
1579,авито тула,126168,Центр


## Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
 
Результат классификации запишите в столбец class


In [6]:
movies=pd.read_csv('../data/movies.csv')
ratings=pd.read_csv('../data/ratings.csv')

In [7]:
mean_rating = ratings.groupby('movieId').mean()

full_movies = movies.merge(mean_rating, left_on='movieId', right_on='movieId', how='inner')[['movieId','title','rating']]

def set_movie_class(df):
    new_df = df
    new_df['class'] = new_df['rating'].apply(lambda row: 'Low' if row <= 2 else 'Hight' if row >= 4.5 else 'Mid' )
    return new_df

set_movie_class(full_movies).head(10)

,movieId,title,rating,class
0,1,Toy Story (1995),3.872470,Mid
1,2,Jumanji (1995),3.401869,Mid
2,3,Grumpier Old Men (1995),3.161017,Mid
3,4,Waiting to Exhale (1995),2.384615,Mid
4,5,Father of the Bride Part II (1995),3.267857,Mid
5,6,Heat (1995),3.884615,Mid
6,7,Sabrina (1995),3.283019,Mid
7,8,Tom and Huck (1995),3.800000,Mid
8,9,Sudden Death (1995),3.150000,Mid
9,10,GoldenEye (1995),3.450820,Mid


## Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.
 

In [8]:
ratings=pd.read_csv('../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
def get_time_diff(x):
    return (datetime.utcfromtimestamp(x[1]) - datetime.utcfromtimestamp(x[0])).days

count_ratings = ratings.groupby('userId').agg(['count', 'min', 'max'])['timestamp']
count_ratings['Lifetime'] = count_ratings[['min', 'max']].apply(get_time_diff, axis=1)

count_ratings.head()


,count,min,max,Lifetime
userId,,,,
1,20,1260759108,1260759205,0
2,76,835355395,835356246,0
3,51,1298861589,1298932787,0
4,204,949778714,949982274,2
5,100,1163373044,1163375145,0


## Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
1. В переменную years запишите список из всех годов с 1950 по 2010.
 
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [10]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
years = [i for i in range(1950, 2011,1)]

def production_year(df):
    new_df = df
    new_df['year'] = df['title'].apply(lambda row: ( int(re.findall('\((\d+)\)', row)[0])
                                                       if int(re.findall('\((\d+)\)', row)[0]) in years else 1900)
                                                           if re.findall('\(\d+\)', row) else 1900)
    return new_df

production_year(full_movies).groupby('year').mean()[['rating']].sort_values(['rating'], ascending = False)
    

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295
